# Federated Learning for Spatio-Temporal Predictions

In [2]:
# NOTE: If you are running a Jupyter notebook, and installing a locally built
# pip package, you may need to edit the following to point to the '.whl' file
# on your local filesystem.

# !pip install --quiet tensorflow_federated
# !pip install --quiet tf-nightly

# NOTE: Jupyter requires a patch to asyncio.
# !pip install --upgrade nest_asyncio
import nest_asyncio
nest_asyncio.apply()

In [3]:
from __future__ import absolute_import, division, print_function

import collections
import functools
import os
import six
import time
import glob
from datetime import datetime
import pickle

import numpy as np
import tensorflow as tf
import tensorflow_federated as tff

tf.compat.v1.enable_v2_behavior()

np.random.seed(0)

# Test the TFF is working:
tff.federated_computation(lambda: 'Hello, World!')()

'Hello, World!'

In [4]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm

In [5]:
import datetime

In [6]:
%load_ext tensorboard

# 1. Data Preprocessing

## 1.1 Samples (indexed by experiment)

In [ ]:
# path = '../data/avoidance_20200131_204454.dat' #10 Robots
path = '../data/avoidance_20200131_204700.dat' #20 Robots
# path = '../data/avoidance_20200131_205449.dat' #40 Robots

samples_central = {}

for filename in glob.iglob(path):
    print(filename)
    exp_id = filename[-10:-4]
    count = 0
    samples_central.update({exp_id : {}})
    samples_central[exp_id].update({count : []})
    for line in open(filename):
        data = line.split(',')
        if len(data) == 7:
            x1 = float(data[3])
            x2 = float(data[4])
            samples_central[exp_id][count].append((x1, x2))
            # assumption that trajectories are 99 steps
            if len(samples_central[exp_id][count]) == 99:
                del samples_central[exp_id][count][98] # easier with even length
                count+=1
                samples_central[exp_id].update({count : []})
    # delete last empty trajectory
    del samples_central[exp_id][count]

## 1.2 Samples (indexed by experiment, client and collection time)

In [7]:
# path = '../data/avoidance_20200131_204454.dat'#'../data/[!G_][!flocking]**.dat'
path = '../data/avoidance_20200131_204700.dat' #20 Robots
# path = '../data/avoidance_20200131_205449.dat' #40 Robots


samples = {} #samples{ <exp_id> : {<rid>: { <traj_id> : { 'traj' : [], 'end' : <time_collected> }}}

for filename in glob.iglob(path):
    print(filename)
    exp_id = filename[-10:-4]
    last_sample_keys = {}
    samples.update({exp_id : {}})
    for line in open(filename):
        data = line.split(',')
        if len(data) == 7:
            rid = int(data[0])
            t = int(data[2])
            x1 = float(data[3])
            x2 = float(data[4])
            if rid in samples[exp_id].keys():
                last_key = last_sample_keys[rid]
                if(last_key not in samples[exp_id][rid]):
                    samples[exp_id][rid][last_key] = {'traj': [], 'end' : 0}
                samples[exp_id][rid][last_key]['traj'].append((x1, x2))
                samples[exp_id][rid][last_key]['end'] = t
                if len(samples[exp_id][rid][last_key]['traj']) == 99:
                    del samples[exp_id][rid][last_key]['traj'][98]
                    samples[exp_id][rid][last_key]['end'] = t
                    last_sample_keys[rid]+=1
            else:
                samples[exp_id].update({rid: {}})
                samples[exp_id][rid].update({0: {'traj': [(x1, x2)], 'end': 0}})
                last_sample_keys.update({rid: 0})
        else:
            last_key = last_sample_keys[rid] 
            if(last_key in samples[exp_id][rid] and len(samples[exp_id][rid][last_key]) != 0):
                last_sample_keys[rid]+=1

../data/avoidance_20200131_204700.dat


## 1.3 Communication graph 

In [ ]:
path = '../data/G_avoidance_20200131_204454.dat'

neighbors = {}  #neigbors{ <exp_id> : {<rid>: { <t> : [<neighbors>]}}

for filename in glob.iglob(path):
    print(filename)    
    exp_id = filename[-10:-4]
    neighbors.update({exp_id : {}})
    for line in open(filename):
        data = line.split(',')
        rid = int(data[0])
        t = int(data[1])
        nid = int(data[2])
        if(t != 0):
            if(rid not in neighbors[exp_id].keys()):
                neighbors[exp_id].update({rid: {}})
            if(t not in neighbors[exp_id][rid].keys()):
                neighbors[exp_id][rid].update({t:[]})
            neighbors[exp_id][rid][t].append(nid)

## 1.4 Format checks

In [ ]:
print(samples.keys())
print(len(samples['205449'][1].keys()))

In [ ]:
for k,v in samples['204454'].items():
    for kk, vv in v.items():
        if len(vv['traj']) != 98:
            print(len(v),len(vv['traj']))

In [ ]:
for k,v in samples_central['204700'].items():
        if len(v) != 98:
            print(k,len(v))

In [ ]:
# print(samples_central['205449'][1])
print(len(samples_central['204700'].keys()))

# 2. Model Definition

## 2.1 Parameters

In [8]:
EMBEDDING_SIZE = 32
DROP_RATE = 0.3
NUM_OUTPUTS = 2

SEQ_LENGTH = 49
DIM_INPUT = 2

In [9]:
sample_shape = (SEQ_LENGTH, DIM_INPUT)

## 2.2 Simple LSTM 

In [21]:
def create_model():
    return tf.keras.models.Sequential([
    tf.keras.layers.Masking(mask_value=-10.,input_shape=sample_shape),
    tf.keras.layers.LSTM(EMBEDDING_SIZE,
                         return_sequences=True,
                         input_shape=sample_shape),
    tf.keras.layers.Dense(NUM_OUTPUTS)
    ])
#     tf.keras.layers.Dropout(DROP_RATE),

## 2.3 Custom loss functions (todo)

In [ ]:
def r3_bivariate_gaussian_loss(true, pred):
    """
    Rank 3 bivariate gaussian loss function
    Returns results of eq # 24 of http://arxiv.org/abs/1308.0850
    :param true: truth values with at least [mu1, mu2, sigma1, sigma2, rho]
    :param pred: values predicted from a model with the same shape requirements as truth values
    :return: the log of the summed max likelihood
    """
    x_coord = true[:, :, 0]
    y_coord = true[:, :, 1]
    mu_x = pred[:, :, 0]
    mu_y = pred[:, :, 1]

    # exponentiate the sigmas and also make correlative rho between -1 and 1.
    # eq. # 21 and 22 of http://arxiv.org/abs/1308.0850
    # analogous to https://github.com/tensorflow/magenta/blob/master/magenta/models/sketch_rnn/model.py#L326
    sigma_x = K.exp(K.abs(pred[:, :, 2]))
    sigma_y = K.exp(K.abs(pred[:, :, 3]))
    rho = K.tanh(pred[:, :, 4]) * 0.1  # avoid drifting to -1 or 1 to prevent NaN, you will have to tweak this multiplier value to suit the shape of your data

    norm1 = K.log(1 + K.abs(x_coord - mu_x))
    norm2 = K.log(1 + K.abs(y_coord - mu_y))

    variance_x = K.softplus(K.square(sigma_x))
    variance_y = K.softplus(K.square(sigma_y))
    s1s2 = K.softplus(sigma_x * sigma_y)  # very large if sigma_x and/or sigma_y are very large

    # eq 25 of http://arxiv.org/abs/1308.0850
    z = ((K.square(norm1) / variance_x) +
         (K.square(norm2) / variance_y) -
         (2 * rho * norm1 * norm2 / s1s2))  # z → -∞ if rho * norm1 * norm2 → ∞ and/or s1s2 → 0
    neg_rho = 1 - K.square(rho)  # → 0 if rho → {1, -1}
    numerator = K.exp(-z / (2 * neg_rho))  # → ∞ if z → -∞ and/or neg_rho → 0
    denominator = (2 * np.pi * s1s2 * K.sqrt(neg_rho)) + epsilon()  # → 0 if s1s2 → 0 and/or neg_rho → 0
    pdf = numerator / denominator  # → ∞ if denominator → 0 and/or if numerator → ∞
    return K.log(K.sum(-K.log(pdf + epsilon())))  # → -∞ if pdf → ∞

# 3. Datasets 

In [ ]:
TRAIN_RATIO = 0.8
VAL_RATIO =  1 - TRAIN_RATIO
PAST_HISTORY = 49
TIME_STEP = 0.1

BATCH_SIZE = 256
BUFFER_SIZE = 10000

## 3.1 Utilities

In [ ]:
def _create_series_examples_from_dict(data_dict, start_index, end_index, history_size):
    data = []
    labels = []
    list_dataset = list(data_dict.values())
    array_dataset = np.asarray(list_dataset)
    for i in range(start_index, end_index):
        data.append(array_dataset[i][:history_size])
        labels.append(array_dataset[i][history_size:])
    data = np.asarray(data).reshape(end_index-start_index, history_size, 2)
    labels = np.asarray(labels).reshape(end_index-start_index, len(list_dataset[0]) - history_size , 2)
    
    return data, labels

In [ ]:
def create_training_and_val_sets(data_dict, past_history=PAST_HISTORY, future_target=PAST_HISTORY):
    
    x_train = np.zeros((1, past_history, DIM_INPUT))
    y_train = np.zeros((1, future_target, DIM_INPUT))
    x_val = np.zeros((1, past_history, DIM_INPUT))
    y_val = np.zeros((1, future_target, DIM_INPUT))

    for v in data_dict.values():
        tot_samples = len(v)
        train_split = round(TRAIN_RATIO * tot_samples)
        x_train_tmp, y_train_tmp = _create_series_examples_from_dict(v, 0, train_split, past_history)
        x_val_tmp, y_val_tmp = _create_series_examples_from_dict(v, train_split, tot_samples, past_history)
        x_train = np.concatenate([x_train, x_train_tmp], axis=0)
        y_train = np.concatenate([y_train, y_train_tmp], axis=0)
        x_val = np.concatenate([x_val, x_val_tmp], axis=0)
        y_val = np.concatenate([y_val, y_val_tmp], axis=0)
        
    return x_train, x_val, y_train, y_val

In [ ]:
def create_datasets(x_train, x_val, y_train, y_val):
    train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    train_set = train_set.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()
    val_set = tf.data.Dataset.from_tensor_slices((x_val, y_val))
    val_set = val_set.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()
    return train_set, val_set

## 3.2 Centralized

In [ ]:
x_train_C, x_val_C, y_train_C, y_val_C = create_training_and_val_sets(samples_central)

In [ ]:
train_set_C, val_set_C = create_datasets(x_train_C, x_val_C, y_train_C, y_val_C)

In [ ]:
print(train_set_C)

# 4. Training

## 4.1 Centralized 

### 4.1.1 Utilities

In [ ]:
# Create new callback
class MyHistory(tf.keras.callbacks.Callback):
    """Adapted from https://github.com/keras-team/keras/blob/master/keras/callbacks/callbacks.py#L614"""

    def on_train_begin(self, logs=None):
        self.epoch = []
        self.times = []
        self.history = {}
        self.start = datetime.datetime.now()

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        self.epoch.append(epoch)
        delta = float((datetime.datetime.now() - self.start).total_seconds())
        self.times.append(delta)
        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)

### 4.1.2 Parameters

In [ ]:
EVALUATION_INTERVAL = 50
EPOCHS = 150  

### 4.1.3 Training loop

In [ ]:
# Instantiate callback
myHistory = MyHistory()

In [ ]:
simple_lstm = create_model()

In [ ]:
logdir = "logs/scalars/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

simple_lstm.compile(optimizer='SGD',
        loss='mean_squared_error')
simple_lstm.fit(train_set_C, epochs=EPOCHS,
              steps_per_epoch=EVALUATION_INTERVAL,
              validation_data=val_set_C, validation_steps=50,
              callbacks=[tensorboard_callback, myHistory])

# Note: with the regularization loss, it's normal that the validation loss is smaller than the training loss!

In [ ]:
tensorboard quit

### 4.1.4 Save Training History

In [ ]:
filehandler = open('Centralized_history_K20_avoid_'+ datetime.datetime.now().strftime("%Y%m%d-%H%M%S"), 'wb') 
data = {'losses' : myHistory.history, 'times' : myHistory.times}
pickle.dump(data, filehandler)
filehandler.close()

del data
del samples_central
del simple_lstm
del myHistory

In [ ]:
file = open('Centralized_history_K20_avoid_20200212-181825','rb')
test = pickle.load(file)
print(test)

## 4.2 Federated Averaging in Server Setting

### 4.2.1 Parameters

In [11]:
EXP_DURATION = 100000
BATCH_SIZE = 60 #DATASET_PARTITION
LOCAL_EPOCHS = 1 # 10 100
TRAIN_FRAC = 0.8
NUM_ROUNDS = 100
ROUND_DURATION = int(EXP_DURATION / NUM_ROUNDS)

TRAIN_RATIO = 0.8
VAL_RATIO = 1 - TRAIN_RATIO
PAST_HISTORY = 49
FUTURE_TARGET = 49

In [12]:
print(samples.keys())

dict_keys(['204700'])


In [13]:
for i in range(1,len(samples['204700'])+1):
    print(len(samples['204700'][i]))

5828
6310
6058
6000
4802
5937
5889
4697
6207
4623
3656
6170
5905
5937
6343
5971
5919
6090
5123
6277


### 4.2.2 Utilities

In [14]:
 def _create_series_examples_from_batch(dataset, start_index, end_index, history_size):
    data = []
    labels = []
    list_dataset = list(dataset)
    array_dataset = np.asarray(list_dataset)
    for i in range(start_index, end_index):
        data.append(array_dataset[i][:history_size])
        labels.append(array_dataset[i][history_size:])
        
    data = np.asarray(data).reshape(end_index-start_index, history_size, 2)
    labels = np.asarray(labels).reshape(end_index-start_index, len(list_dataset[0]) - history_size , 2)
    
    return data, labels

In [15]:
def create_training_and_val_batch(batch, past_history=PAST_HISTORY, future_target=PAST_HISTORY):
    
    x_train = np.zeros((1,PAST_HISTORY,2))
    y_train = np.zeros((1,FUTURE_TARGET,2))
    x_val = np.zeros((1,PAST_HISTORY,2))
    y_val = np.zeros((1,FUTURE_TARGET,2))
    for v in batch:
        tot_samples = len(v)
        train_split = round(TRAIN_RATIO * tot_samples)
#         print(len(v), train_split)
        x_train_tmp, y_train_tmp = _create_series_examples_from_batch(v, 0, train_split, PAST_HISTORY)
        x_val_tmp, y_val_tmp = _create_series_examples_from_batch(v, train_split, tot_samples, PAST_HISTORY)
        x_train = np.concatenate([x_train, x_train_tmp], axis=0)
        y_train = np.concatenate([y_train, y_train_tmp], axis=0)
        x_val = np.concatenate([x_val, x_val_tmp], axis=0)
        y_val = np.concatenate([y_val, y_val_tmp], axis=0)
        
    return x_train, x_val, y_train, y_val

In [16]:
def create_datasets_FA(x_train, x_val, y_train, y_val):
    train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    train_set = train_set.cache().batch(BATCH_SIZE).repeat()
    val_set = tf.data.Dataset.from_tensor_slices((x_val, y_val))
    val_set = val_set.cache().batch(BATCH_SIZE).repeat()
    return train_set, val_set

In [17]:
def average_weights(weights):
    avg = np.zeros_like(trainable_weights[1][1])
    count = 0
    for k,v in weights.items():
        if(len(v) != 0):
            avg = np.add(avg, v)
            count += 1
    if(count != 0):
        avg = avg/count
    avg = np.squeeze(avg)
    return avg           

In [18]:
# Create new callback
class LocalHistory(tf.keras.callbacks.Callback):
    """Adapted from https://github.com/keras-team/keras/blob/master/keras/callbacks/callbacks.py#L614"""

    def on_train_begin(self, logs=None):
        self.epoch = []
        self.times = []
        self.history = {}
        self.start = datetime.datetime.now()

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        self.epoch.append(epoch)
        delta = float((datetime.datetime.now() - self.start).total_seconds())
        self.times.append(delta)
        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)

### 4.2.3 Rounds by global clock and data 

In [19]:
from tensorflow.keras.models import load_model

In [ ]:
history = {}
local_lstm = create_model()

# Load the pre-trained LSTM
#   #tf.keras.models.load_model('pretrained_lstm.h5', compile=False)

# For each experiment
for exp in samples.keys():
    
    history.update({exp : {}})
    
    # Per experiment settings 
    num_robots = len(samples[exp].keys())
    
    for i in range(1, num_robots + 1):
        history[exp].update({i : {}})
    
    # Un-synch clocks by random integer (todo)
    time_offsets = np.random.randint(0, 10, size=(num_robots,))
    # Set global clock to 0
    global_clock = 0
    round_num = 1
    
    # Initialize weights
    trainable_weights = {}
    w  = [v.numpy() for v in local_lstm.trainable_weights]
    w_list = [[w] for i in range(num_robots)]
    trainable_weights[0] = {}
    trainable_weights[round_num] = {k : v for (k, v) in zip(range(1, num_robots + 1), w_list)}
    
    leftover_samples_buffer = {}
    
    # buffer of last data index of previous round for each robot
    last_idx_previous_round = np.zeros(num_robots, dtype=int)
    
    for t in range(ROUND_DURATION, EXP_DURATION, ROUND_DURATION):
        
        print("_______________________")
        print("Round ", round_num)
        print("_______________________")
        
        # One round for each robot: data collection, local training and global update
        for i in samples[exp].keys():
            
            print("Robot ", i)
            
            batch = []
            
            # Get data index at time of round
            cond = True
            current_idx =  last_idx_previous_round[i-1]
            while(cond):
                current_idx+=1
                if(samples[exp][i][current_idx]['end'] > t):
                    cond = False
            
            num_samples = current_idx - last_idx_previous_round[i-1]
            
            print("Num samples", num_samples)
            
             # Check that we have enough data collected to participate in the round
            if(num_samples >= BATCH_SIZE):
                # Take extra data collected before end of round
                tmp = [samples[exp][i][j]['traj'] for j in range(last_idx_previous_round[i-1], current_idx)]
                batch.append(tmp)
                %xdel tmp
                # Take remaining data from previous round
                if(i in leftover_samples_buffer.keys() and len(leftover_samples_buffer[i]) > 0):
                    batch.append(leftover_samples_buffer[i])
                    leftover_samples_buffer[i] = []
            # Check that we have enough previous data      
            elif(i in leftover_samples_buffer.keys() 
                 and (len(leftover_samples_buffer[i]) + num_samples) >= BATCH_SIZE): 
                batch.append(leftover_samples_buffer[i])
                leftover_samples_buffer[i] = []
            else:
                if(i not in leftover_samples_buffer.keys()):
                    leftover_samples_buffer.update({i: []})
                leftover_samples_buffer[i] = [samples[exp][i][j]['traj'] for j in range(last_idx_previous_round[i-1], current_idx)]
                last_idx_previous_round[i-1] = current_idx
                print('Robot not participating', i)
                continue
            last_idx_previous_round[i-1] = current_idx
            
            # Get weights
            current_weights = average_weights(trainable_weights[round_num])
            
#             # Perform local training
            
            # Create datasets
            x_train_FA, x_val_FA, y_train_FA, y_val_FA = create_training_and_val_batch(batch)
            train_batch, val_batch = create_datasets_FA(x_train_FA, x_val_FA, y_train_FA, y_val_FA)
#             # Clone simple_lstm and initialize it with newest weights
            local_lstm = tf.keras.models.load_model('lstm.h5', compile=False)
            keras_model_clone = tf.keras.models.clone_model(local_lstm)
            keras_model_clone.compile(optimizer='SGD', loss='mean_absolute_error')
            keras_model_clone.set_weights(current_weights)
            l_history = LocalHistory()
            keras_model_clone.fit(train_batch, epochs=LOCAL_EPOCHS,
              steps_per_epoch=len(x_train_FA),
              validation_data=val_batch, 
              validation_steps=len(x_val_FA),
              callbacks=[l_history])
#             # Write weights 
            if((round_num+1) not in trainable_weights.keys()):
                trainable_weights.update({(round_num+1): {}})
            trainable_weights[round_num+1].update({i: keras_model_clone.get_weights()})
            # Write metrics
            history[exp][i].update({round_num : l_history})
            %xdel current_weights
            %xdel l_history          
            %xdel train_batch
            %xdel val_batch
            %xdel batch
            %xdel x_train_FA
            %xdel x_val_FA
            %xdel y_train_FA
            %xdel y_val_FA
            %xdel keras_model_clone
            %xdel local_lstm
            tf.keras.backend.clear_session()
        round_num+=1

Round  1
Robot  1
Num samples 57
Robot not participating 1
Robot  10
Num samples 30
Robot not participating 10
Robot  11
Num samples 40
Robot not participating 11
Robot  12
Num samples 62
Train for 51 steps, validate for 13 steps
51/51 [==============================] - 3s 55ms/step - loss: 0.8072 - val_loss: 0.7355
NameError: name 'train_batch, val_batch' is not defined
Robot  13
Num samples 87
Train for 71 steps, validate for 18 steps
71/71 [==============================] - 4s 52ms/step - loss: 0.7567 - val_loss: 0.4517
NameError: name 'train_batch, val_batch' is not defined
Robot  14
Num samples 80
Train for 65 steps, validate for 17 steps
65/65 [==============================] - 3s 47ms/step - loss: 0.7836 - val_loss: 0.5245
NameError: name 'train_batch, val_batch' is not defined
Robot  15
Num samples 80
Train for 65 steps, validate for 17 steps
65/65 [==============================] - 4s 57ms/step - loss: 0.7261 - val_loss: 0.5727
NameError: name 'train_batch, val_batch' is not d

38/38 [==============================] - 3s 82ms/step - loss: 0.3049 - val_loss: 0.2162
NameError: name 'tmp' is not defined
NameError: name 'train_batch, val_batch' is not defined
Robot  3
Num samples 49
Robot not participating 3
Robot  4
Num samples 56
Robot not participating 4
Robot  5
Num samples 38
Robot not participating 5
Robot  6
Num samples 84
Train for 101 steps, validate for 26 steps
101/101 [==============================] - 5s 45ms/step - loss: 0.2164 - val_loss: 0.1954
NameError: name 'train_batch, val_batch' is not defined
Robot  7
Num samples 50
Robot not participating 7
Robot  8
Num samples 72
Train for 59 steps, validate for 15 steps
59/59 [==============================] - 5s 90ms/step - loss: 0.1961 - val_loss: 0.1891
NameError: name 'train_batch, val_batch' is not defined
Robot  9
Num samples 89
Train for 72 steps, validate for 19 steps
72/72 [==============================] - 4s 62ms/step - loss: 0.2196 - val_loss: 0.2137
NameError: name 'train_batch, val_batch' i

Train for 83 steps, validate for 22 steps
83/83 [==============================] - 4s 44ms/step - loss: 0.1371 - val_loss: 0.1337
NameError: name 'train_batch, val_batch' is not defined
Robot  14
Num samples 36
Robot not participating 14
Robot  15
Num samples 90
Train for 73 steps, validate for 19 steps
73/73 [==============================] - 3s 47ms/step - loss: 0.1363 - val_loss: 0.1332
NameError: name 'train_batch, val_batch' is not defined
Robot  16
Num samples 93
Train for 121 steps, validate for 31 steps
121/121 [==============================] - 5s 40ms/step - loss: 0.1235 - val_loss: 0.1413
NameError: name 'train_batch, val_batch' is not defined
Robot  17
Num samples 92
Train for 114 steps, validate for 29 steps
114/114 [==============================] - 5s 44ms/step - loss: 0.1385 - val_loss: 0.1257
NameError: name 'train_batch, val_batch' is not defined
Robot  18
Num samples 88
Train for 71 steps, validate for 19 steps
71/71 [==============================] - 4s 51ms/step - 

Train for 63 steps, validate for 17 steps
63/63 [==============================] - 3s 51ms/step - loss: 0.1691 - val_loss: 0.2210
NameError: name 'train_batch, val_batch' is not defined
Robot  3
Num samples 81
Train for 105 steps, validate for 27 steps
105/105 [==============================] - 5s 44ms/step - loss: 0.1566 - val_loss: 0.1567
NameError: name 'train_batch, val_batch' is not defined
Robot  4
Num samples 47
Robot not participating 4
Robot  5
Num samples 67
Train for 55 steps, validate for 14 steps
55/55 [==============================] - 3s 57ms/step - loss: 0.1728 - val_loss: 0.1373
NameError: name 'train_batch, val_batch' is not defined
Robot  6
Num samples 42
Robot not participating 6
Robot  7
Num samples 91
Train for 117 steps, validate for 30 steps
117/117 [==============================] - 5s 41ms/step - loss: 0.1660 - val_loss: 0.1639
NameError: name 'train_batch, val_batch' is not defined
Robot  8
Num samples 60
Train for 49 steps, validate for 13 steps
49/49 [=====

Train for 76 steps, validate for 20 steps
76/76 [==============================] - 4s 50ms/step - loss: 0.1406 - val_loss: 0.1588
NameError: name 'train_batch, val_batch' is not defined
Robot  10
Num samples 70
Train for 57 steps, validate for 15 steps
57/57 [==============================] - 3s 58ms/step - loss: 0.1403 - val_loss: 0.1189
NameError: name 'train_batch, val_batch' is not defined
Robot  11
Num samples 37
Robot not participating 11
Robot  12
Num samples 103
Train for 83 steps, validate for 22 steps
83/83 [==============================] - 4s 43ms/step - loss: 0.1361 - val_loss: 0.1546
NameError: name 'train_batch, val_batch' is not defined
Robot  13
Num samples 78
Train for 63 steps, validate for 17 steps
63/63 [==============================] - 3s 55ms/step - loss: 0.1389 - val_loss: 0.1508
NameError: name 'train_batch, val_batch' is not defined
Robot  14
Num samples 54
Train for 36 steps, validate for 10 steps
36/36 [==============================] - 3s 75ms/step - loss:

110/110 [==============================] - 5s 44ms/step - loss: 0.1372 - val_loss: 0.1215
NameError: name 'train_batch, val_batch' is not defined
Robot  14
Num samples 46
Robot not participating 14
Robot  15
Num samples 71
Train for 58 steps, validate for 15 steps
58/58 [==============================] - 4s 65ms/step - loss: 0.1168 - val_loss: 0.0957
NameError: name 'train_batch, val_batch' is not defined
Robot  16
Num samples 58
Robot not participating 16
Robot  17
Num samples 56
Robot not participating 17
Robot  18
Num samples 72
Train for 59 steps, validate for 15 steps
59/59 [==============================] - 4s 63ms/step - loss: 0.1203 - val_loss: 0.1027
NameError: name 'train_batch, val_batch' is not defined
Robot  19
Num samples 72
Train for 59 steps, validate for 15 steps
59/59 [==============================] - 3s 58ms/step - loss: 0.1245 - val_loss: 0.1117
NameError: name 'train_batch, val_batch' is not defined
Robot  2
Num samples 80
Train for 65 steps, validate for 17 steps

94/94 [==============================] - 4s 42ms/step - loss: 0.1353 - val_loss: 0.1312
NameError: name 'train_batch, val_batch' is not defined
Robot  20
Num samples 85
Train for 115 steps, validate for 30 steps
115/115 [==============================] - 5s 42ms/step - loss: 0.1331 - val_loss: 0.1209
NameError: name 'train_batch, val_batch' is not defined
Robot  3
Num samples 83
Train for 67 steps, validate for 18 steps
67/67 [==============================] - 3s 48ms/step - loss: 0.1189 - val_loss: 0.0979
NameError: name 'train_batch, val_batch' is not defined
Robot  4
Num samples 70
Train for 57 steps, validate for 15 steps
57/57 [==============================] - 3s 59ms/step - loss: 0.1369 - val_loss: 0.0928
NameError: name 'train_batch, val_batch' is not defined
Robot  5
Num samples 34
Train for 42 steps, validate for 11 steps
42/42 [==============================] - 3s 72ms/step - loss: 0.1138 - val_loss: 0.1351
NameError: name 'tmp' is not defined
NameError: name 'train_batch, v

Train for 34 steps, validate for 9 steps
34/34 [==============================] - 3s 80ms/step - loss: 0.0962 - val_loss: 0.1315
NameError: name 'tmp' is not defined
NameError: name 'train_batch, val_batch' is not defined
Robot  12
Num samples 64
Train for 52 steps, validate for 14 steps
52/52 [==============================] - 3s 58ms/step - loss: 0.1128 - val_loss: 0.1429
NameError: name 'train_batch, val_batch' is not defined
Robot  13
Num samples 58
Train for 31 steps, validate for 9 steps
31/31 [==============================] - 2s 80ms/step - loss: 0.1015 - val_loss: 0.1718
NameError: name 'tmp' is not defined
NameError: name 'train_batch, val_batch' is not defined
Robot  14
Num samples 81
Train for 66 steps, validate for 17 steps
66/66 [==============================] - 4s 54ms/step - loss: 0.1245 - val_loss: 0.1318
NameError: name 'train_batch, val_batch' is not defined
Robot  15
Num samples 47
Robot not participating 15
Robot  16
Num samples 46
Robot not participating 16
Robot

52/52 [==============================] - 3s 61ms/step - loss: 0.1070 - val_loss: 0.1237
NameError: name 'train_batch, val_batch' is not defined
Robot  19
Num samples 57
Train for 47 steps, validate for 12 steps
47/47 [==============================] - 3s 64ms/step - loss: 0.1123 - val_loss: 0.1267
NameError: name 'tmp' is not defined
NameError: name 'train_batch, val_batch' is not defined
Robot  2
Num samples 67
Train for 55 steps, validate for 14 steps
55/55 [==============================] - 3s 58ms/step - loss: 0.1132 - val_loss: 0.1772
NameError: name 'train_batch, val_batch' is not defined
Robot  20
Num samples 73
Train for 93 steps, validate for 24 steps
93/93 [==============================] - 4s 45ms/step - loss: 0.1126 - val_loss: 0.1207
NameError: name 'train_batch, val_batch' is not defined
Robot  3
Num samples 61
Train for 50 steps, validate for 13 steps
50/50 [==============================] - 154s 3s/step - loss: 0.1139 - val_loss: 0.0980
NameError: name 'train_batch, val

### 4.2.4 Save training data

In [ ]:
data = {}
for exp in history:
    data.update({exp : {} })
    for round_num in range(1, NUM_ROUNDS):
        data[exp].update({round_num: {}})
        for robot in history[exp]:
            if (round_num in history[exp][robot].keys()):
                data[exp][round_num].update({robot : {}})
                data[exp][round_num][robot] = {'losses' : history[exp][robot][round_num].history, 'times' : history[exp][robot][round_num].times}

In [ ]:
filehandler = open('FA_history_' + str(num_robots) + '_' + str(BATCH_SIZE) + '_ '+ datetime.datetime.now().strftime("%Y%m%d-%H%M%S"), 'wb') 
pickle.dump(data, filehandler)
filehandler.close()
filehandler.close()

## 4.3 Serverless Federated Averaging

In [ ]:
class VSElem():
    def __init__(self, data, t, i):
        self.data = data
        self.timestamp = t
        self.rid = i

class VirtualStigmery():
    def __init__(self, vid):
        self.id = vid
        self.dict = {}
        
#     def put():
    
    
#     def __call__():

# 1) vstig : key = round_num, data = < num_writes, weights_average > and store last 
#    round participated in (but conflicts problematic)
# 2) vstig : key = round_num, data = < rid_list, weights > and last round participated in (but increased load)
# 3) vstig : id = round_num, key = rid, data = < weights > and last round participated in (but delay in getting
# the newest round? or possible to start a wrong round)
# Furthermore, to avoid conflicts, do we stop at exactly a certain number of participants for the weight average?
# one vstig (1-2) or one per round (3) ? or CRDT after all ?
        

# 5. Inference and Evaluation

## 5.1 Validation Loss Curve

### 5.1.1 Processing for FA

In [ ]:
avg_val_loss = np.zeros(NUM_ROUNDS)

for exp in history:
    for round_num in range(1, NUM_ROUNDS):
        count = 0.
        sum_loss = 0.
        for robot in history[exp]:
            if (round_num in history[exp][robot].keys()):
                count +=1
                sum_loss += history[exp][robot][round_num].history['val_loss'][0]
        if(count != 0):
            avg_val_loss[round_num - 1] = sum_loss/count
        

### 5.1.2 Plot

In [ ]:
fig = plt.figure(figsize=(12, 6))
ax = plt.gca()
loss_C = myHistory.history['val_loss']
epochs_C = myHistory.epoch
plt.plot(epochs_C[0:50], loss_C[0:50], 'bv-', label='Centralized')
plt.plot(avg_val_loss[avg_val_loss!=0], 'r*-', label='FA')
plt.legend(loc='upper right', fontsize=16)
plt.xlabel('Epochs (Iterations or Communication rounds)', FontSize=16)
plt.ylabel('Validation Loss', FontSize=16)

## 5.2 Runtime Curve

In [ ]:
fig = plt.figure(figsize=(12, 6))
# ax = plt.gca()
# runtimes_C = np.diff(myHistory.times)#.total_seconds() * 1000)
# runtimes_C_s = [float(v.total_seconds())  for v in runtimes_C] 
runtimes_C_s = myHistory.times
epochs_C = myHistory.epoch[1:]
plt.plot(epochs_C, runtimes_C_s, 'b.-', label='Centralized')
plt.legend(loc='upper right')
plt.xlabel('Epochs (Iterations or Communication rounds)')
plt.ylabel('Duration (s)')

## 5.3 Predictions

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

def create_time_steps(length):
    return list(range(-length, 0))

In [ ]:
def multi_step_plot(history, true_future, prediction):

    history=np.array(history[history!=-10]).reshape([-1, 2])
    prediction=np.array(prediction[true_future!=-10]).reshape([-1,2])
    true_future=np.array(true_future[true_future!=-10]).reshape([-1, 2])
    
    fig = plt.figure(figsize=(12, 6))
    ax = plt.gca()
    ax.set_xlim([-10,10])
    ax.set_ylim([-2,2])
    
    num_in = create_time_steps(len(history))
    num_out = len(true_future)
    plt.plot(num_in, history[:, 0], 'b', label='History_x')
    plt.plot(num_in, history[:, 1], 'b', label='History_y')
    plt.plot(np.arange(num_out), true_future, 'bo-',
           label='True Future')
    if prediction.any():
        plt.plot(np.arange(num_out), prediction, 'ro',
                 label='Predicted Future')
        plt.legend(loc='upper left')
        plt.show()

In [ ]:
def multi_step_plot_3D(history, true_future, prediction):
    history=np.array(history[history!=-10]).reshape([-1, 2])
    prediction=np.array(prediction[true_future!=-10]).reshape([-1,2])
    true_future=np.array(true_future[true_future!=-10]).reshape([-1, 2])
    
    fig = plt.figure(figsize=(12, 6))
    ax = Axes3D(fig)
    ax.set_xlim([-2,2])
    ax.set_ylim([-2,2])
    ax.set_zlim([-10,10])
    num_in = create_time_steps(len(history))
    num_out = len(true_future)
    ax.plot3D(np.array(history[:, 0]), np.array(history[:, 1]), num_in, 'bo-', label='History')
    ax.plot3D(np.array(true_future[:,0]), np.array(true_future[:,1]), np.arange(num_out), 'bo',
           label='True Future')
    if prediction.any():
        ax.plot3D(np.array(prediction[:,0]), np.array(prediction[:,1]), np.arange(num_out), 'ro',
                 label='Predicted Future')
        plt.legend(loc='upper left')
        plt.show()

In [ ]:
# Stanford validation set
for x, y in val_set.take(3):
    multi_step_plot_3D(history=x[0], true_future=y[0], prediction=simple_lstm_model.predict(x)[0])
    multi_step_plot(history=x[0], true_future=y[0], prediction=simple_lstm_model.predict(x)[0])

# OLD CODE

# 2. Client-partioned Federated Learning from Trajectory Data

## 2.1 Load Distributed Data

In [ ]:
path = '../data/short_data/avoidance_20200127_1615**.dat'#'../data/[!G_][!flocking]**.dat' #foraging_20191211_213418.dat' #foraging**.dat'

samples = {}
last_sample_keys = {}

for filename in glob.iglob(path):
    print(filename)
    for line in open(filename):
        data = line.split(',')
        if len(data) == 7:
            rid = data[0]
            x1 = float(data[3])
            x2 = float(data[4])
            if rid in samples.keys():
                last_key = last_sample_keys[rid]
                if(last_key not in samples[rid]):
                    samples[rid][last_key] = []
                samples[rid][last_key].append((x1, x2))
                if len(samples[rid][last_key]) == 99:
                    del samples[rid][last_key][98]
                    last_sample_keys[rid]+=1
            else:
                samples.update({rid: {}})
                samples[rid].update({0: [(x1, x2)]})
                last_sample_keys.update({rid: 0})
        else:
            last_key = last_sample_keys[rid] 
            if(last_key in samples[rid] and len(samples[rid][last_key]) != 0):
                last_sample_keys[rid]+=1

## 2.3 Visualize Data

In [ ]:
TIME_STEP = 0.1

colors = cm.rainbow(np.linspace(0, 1, len(samples.keys())))
cnt = 0

plt.figure()
for v in samples.values():
    for traj in v.values():
        if (len(traj) > 0):
            x, y = zip(*traj)
        plt.plot(x, y, color=colors[cnt])
    cnt += 1
    break
    
ax = plt.gca()
circle2 = plt.Circle((0, 0), 2, color='b', fill=False)
ax.add_artist(circle2)
plt.axis('square')
plt.show( )

In [ ]:
count_traj = sum(len(v) for v in samples.values())

print(count_traj)

In [ ]:
for k,v in samples.items():
    for kk, vv in v.items():
        if len(vv) != 98:
            print(len(v),len(vv))

## 2.4 Create Labels

In [ ]:
def tuple_list_data(dataset, start_index, end_index, history_size):
    data = []
    labels = []
    list_dataset = list(dataset.values())
    for i in range(start_index, end_index):
        data.append(np.asarray(list_dataset)[i][:history_size])
        labels.append(np.asarray(list_dataset)[i][history_size:])
        
    data = np.asarray(data).reshape(end_index-start_index, history_size, 2)
    labels = np.asarray(labels).reshape(end_index-start_index, len(list_dataset[0]) - history_size , 2)
    
    return data, labels

## 2.5 Create tff Dataset and tf Dataset Per Client

In [ ]:
TRAIN_RATIO = 0.2
VAL_RATIO = 1 - TRAIN_RATIO
PAST_HISTORY = 49
TIME_STEP = 0.1

BATCH_SIZE = 256
BUFFER_SIZE = 10000

In [ ]:
count = 1
dict_train_dataset = {}
dict_val_dataset = {}
for v in samples.values():
    tot_samples = round(len(v)/5)
    train_split = round(TRAIN_RATIO * tot_samples)
    x_train, y_train = tuple_list_data(v, 0, train_split, PAST_HISTORY)
    x_val, y_val = tuple_list_data(v, train_split, tot_samples, PAST_HISTORY)
    dict_train_dataset.update({count: (x_train, y_train)})
    dict_val_dataset.update({count: (x_val, y_val)})
    count+=1
    
#     train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train))
#     train_set = train_set.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()
#     val_set = tf.data.Dataset.from_tensor_slices((x_val, y_val))
#     val_set = val_set.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

In [ ]:
print(x_train.shape, y_train.shape)

In [ ]:
tff_train_set = tff.simulation.FromTensorSlicesClientData(dict_train_dataset)
tff_val_set = tff.simulation.FromTensorSlicesClientData(dict_val_dataset)

In [ ]:
filehandler = open('tff_train_set_'+ datetime.now().strftime("%Y%m%d-%H%M%S"), 'wb') 
pickle.dump(tff_train_set, filehandler)
filehandler = open('tff_val_set_'+ datetime.now().strftime("%Y%m%d-%H%M%S"), 'wb') 
pickle.dump(tff_val_set, filehandler)

In [ ]:
print(tff_train_set.client_ids)
print(tff_train_set._output_shapes)
# print(vars(tff_train_set))

In [ ]:
def preprocess(dataset):
    return (
      dataset.cache()
      #form minibatches
      .batch(BATCH_SIZE))

In [ ]:
example_dataset = train_set

In [ ]:
print(example_dataset)
print(tf.data.experimental.get_structure(example_dataset))

In [ ]:
def data(client, source=tff_train_set):
    return preprocess(source.create_tf_dataset_for_client(client)).take(2)

clients = [1, 2, 3]

train_datasets = [data(client) for client in clients]

# We concatenate the test datasets for evaluation with Keras.
test_dataset = functools.reduce(
    lambda d1, d2: d1.concatenate(d2),
    [data(client, tff_val_set) for client in clients])

# NOTE: If the statement below fails, it means that you are
# using an older version of TFF without the high-performance
# executor stack. Call `tff.framework.set_default_executor()`
tff.framework.set_default_executor(tff.framework.create_local_executor())

## Serialize the keras model into a tff model

In [ ]:
BATCH_SIZE=16
SEQ_LENGTH=49

In [ ]:
#del simple_lstm_model
simple_lstm_model = tf.keras.models.load_model('pretrained_lstm.h5', compile=False)

In [ ]:
def tff_model():
    x = tf.constant(np.random.uniform(-2., 2., size=[BATCH_SIZE, SEQ_LENGTH, 2]))
    dummy_batch = collections.OrderedDict([('x', x), ('y', x)]) 
    keras_model_clone =tf.keras.models.clone_model(simple_lstm_model)
    keras_model_clone.compile(optimizer='adam', loss='mean_absolute_error')
    return tff.learning.from_compiled_keras_model(keras_model_clone,dummy_batch)

In [ ]:
# This command builds all the TensorFlow graphs and serializes them: 
fed_avg = tff.learning.build_federated_averaging_process(model_fn=tff_model)

In [ ]:
state = fed_avg.initialize()

In [ ]:
state = tff.learning.state_with_new_model_weights(
    state,
    trainable_weights=[v.numpy() for v in simple_lstm_model.trainable_weights],
    non_trainable_weights=[
        v.numpy() for v in simple_lstm_model.non_trainable_weights
    ])

In [ ]:
NUM_ROUNDS=3
simple_lstm_model.compile(optimizer='adam', loss='mean_absolute_error')

def keras_evaluate(state, round_num):
    tff.learning.assign_weights_to_keras_model(simple_lstm_model, state.model)
    print('Evaluating before training round', round_num)
    simple_lstm_model.evaluate(example_dataset, steps=2)

for round_num in range(NUM_ROUNDS):
    keras_evaluate(state, round_num)
    # N.B. The TFF runtime is currently fairly slow,
    # expect this to get significantly faster in future releases.
    state, metrics = fed_avg.next(state, train_datasets)
    print('Training metrics: ', metrics)

keras_evaluate(state, NUM_ROUNDS + 1)

# 3. Continuous Data Collection and Multi-Hop Network

## 3.3 Simulation

In [ ]:
print(neighbors['205449'][1][3])

# 4. Graphs

## 4.1 Centralized Version

## Average Consensus

# Suggested extensions

This tutorial is just the first step! Here are some ideas for how you might try extending this notebook:
  * Write a more realistic training loop where you sample clients to train on randomly.
  * Use "`.repeat(NUM_EPOCHS)`" on the client datasets to try multiple epochs of local training (e.g., as in [McMahan et. al.](https://arxiv.org/abs/1602.05629)). See also [Federated Learning for Image Classification](federated_learning_for_image_classification.md) which does this.
  * Change the `compile()` command to experiment with using different optimization algorithms on the client.
  * Try the `server_optimizer` argument to `build_federated_averaging_process` to try different algorithms for applying the model updates on the server.
  * Try the `client_weight_fn` argument to to `build_federated_averaging_process` to try different weightings of the clients. The default weights client updates by the number of examples on the client, but you can do e.g. `client_weight_fn=lambda _: tf.constant(1.0)`.

## Redis

In [ ]:
import redis

In [ ]:
r = redis.Redis(host='localhost', port=6379, db=0)

In [ ]:
# r.keys()
r.set('foo', 'bar')

In [ ]:
r.get('foo')

In [ ]:
%timeit r.set('foo', 'bar')